In [5]:
import sys
sys.path.append('../../')
sys.path.append('../')

import fm
from torch.profiler import profile, record_function, ProfilerActivity


In [6]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
import torch.fx as fx
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [7]:
import time
import torch._dynamo as dynamo


In [4]:
def gen_pattern_replace_and_matcher_for_FM_reduce1(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)

    def forward(self,x):
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      embed = self.embed(x)
      redency_embed = embed[redency_part_slice] 
      unredency_embed = embed[unredency_part_slice] 
      redency_embed_sum = torch.sum(redency_embed,dim=0)
      unredency_embed_sum = torch.sum(unredency_embed,dim=1)
      square_of_sum = (redency_embed_sum + unredency_embed_sum) ** 2
      sum_of_square = torch.sum(embed ** 2,dim=1)
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [67]:
def workload_FM_reduce1(num_field, prefix,dim = 64):
  # print(f"now gen workload of wdl with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  model_ori = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  ori_traced = symbolic_trace(model_ori)
  
  model_modify = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  modify_traced = symbolic_trace(model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM_reduce1(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      )
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [68]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [69]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_FM_reduce1):
  def run(model):
    traced_model = torch.jit.trace(model, torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long))
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = 0
    total_time = []
    t = torch.randint(low=0, high=88, size=(batch ,num_field), dtype=torch.long)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        with record_function("model_inference"):
          for i in range(100):
              start_time = time.time()  # 开始计时
              with torch.no_grad():
                soutput = compiled_model(t)
              end_time = time.time()  # 结束计时
              
              # 计算并打印函数执行所需的时间
              elapsed_time = end_time - start_time
              total_time.append(elapsed_time * 1000)
    print(calculate_mean_and_variance_manual(total_time))
  print(f"now gen workload of fm with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim)
  run(ori)
  run(modify)

In [70]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :1024


STAGE:2025-02-07 20:04:08 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:08 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:08 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9221982955932617, 0.1963539204780318)


STAGE:2025-02-07 20:04:09 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:09 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:09 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9056496620178223, 0.1058287719558848)


In [71]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :2048


STAGE:2025-02-07 20:04:09 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:10 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:10 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0509753227233887, 0.08340551391370354)


STAGE:2025-02-07 20:04:10 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:10 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:10 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.099252700805664, 0.08868707595865999)


In [72]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :4096


STAGE:2025-02-07 20:04:10 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:11 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:11 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.617271900177002, 0.18122039938930357)


STAGE:2025-02-07 20:04:11 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:11 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:11 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.1252918243408203, 0.3433405939631484)


In [73]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :8192


STAGE:2025-02-07 20:04:12 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:12 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:12 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.2750005722045898, 0.22652618852134765)


STAGE:2025-02-07 20:04:12 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:12 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:12 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.3448858261108398, 0.3367337490317368)


In [74]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :1024


STAGE:2025-02-07 20:04:13 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:13 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:13 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9659194946289062, 0.06108847574068932)


STAGE:2025-02-07 20:04:13 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:14 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:14 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9805941581726074, 0.10028525832126434)


In [75]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :2048


STAGE:2025-02-07 20:04:14 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:14 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:14 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9278750419616699, 0.06255252434925751)


STAGE:2025-02-07 20:04:15 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:15 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:15 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.996551513671875, 0.13302601100804168)


In [76]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :4096


STAGE:2025-02-07 20:04:15 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:15 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:15 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.2411952018737793, 0.14192630303000442)


STAGE:2025-02-07 20:04:16 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:16 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection


(1.28767728805542, 0.17845468971131595)


STAGE:2025-02-07 20:04:16 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [77]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :8192


STAGE:2025-02-07 20:04:16 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:17 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:17 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.1631598472595215, 0.3482404149224294)


STAGE:2025-02-07 20:04:17 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:18 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:18 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(13.188176155090332, 0.3715262796276875)


In [78]:
gen_and_test(num_field = 34,prefix = 29, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 34, prefix: 29, batch :1024


STAGE:2025-02-07 20:04:19 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:19 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:19 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9094405174255371, 0.050859313722639854)


STAGE:2025-02-07 20:04:19 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:19 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:19 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.8609294891357422, 0.08673561637806415)


In [79]:
gen_and_test(num_field = 34,prefix = 29, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 34, prefix: 29, batch :2048


STAGE:2025-02-07 20:04:20 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:20 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:20 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9444642066955566, 0.1342105561604967)


STAGE:2025-02-07 20:04:20 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:20 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:20 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9727883338928223, 0.19623622626454562)


In [80]:
gen_and_test(num_field = 34,prefix = 29, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 34, prefix: 29, batch :4096


STAGE:2025-02-07 20:04:21 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:21 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:21 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.080317497253418, 0.08150507185291644)


STAGE:2025-02-07 20:04:22 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:22 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:22 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1264276504516602, 0.23350949070390925)


In [81]:
gen_and_test(num_field = 34,prefix = 29, batch = 8192, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 34, prefix: 29, batch :8192


STAGE:2025-02-07 20:04:22 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:23 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:23 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
STAGE:2025-02-07 20:04:23 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(1.6370749473571777, 0.3104480423360201)


STAGE:2025-02-07 20:04:23 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:23 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.4743781089782715, 0.30109305100154415)


In [82]:
gen_and_test(num_field = 34,prefix = 29, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :1024


STAGE:2025-02-07 20:04:24 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:24 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:24 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0626029968261719, 0.08127375881485932)


STAGE:2025-02-07 20:04:24 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:24 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:24 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9855031967163086, 0.08799419845217926)


In [83]:
gen_and_test(num_field = 34,prefix = 29, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :2048


STAGE:2025-02-07 20:04:25 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:25 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:25 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0665202140808105, 0.07897910074348147)


STAGE:2025-02-07 20:04:25 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:25 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:25 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0752272605895996, 0.11434115669430867)


In [84]:
gen_and_test(num_field = 34,prefix = 29, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :4096


STAGE:2025-02-07 20:04:26 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:26 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:26 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.585986614227295, 0.22572190220557786)


STAGE:2025-02-07 20:04:26 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:26 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:26 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.573479175567627, 0.22574380338369338)


In [85]:
gen_and_test(num_field = 34,prefix = 29, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :8192


STAGE:2025-02-07 20:04:27 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:29 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:29 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(20.481374263763428, 0.30896100826680595)


STAGE:2025-02-07 20:04:30 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:32 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:32 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(20.51579713821411, 1.462477009107488)


In [86]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :1024


STAGE:2025-02-07 20:04:32 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:32 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:32 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.3286662101745605, 0.12576482965300784)


STAGE:2025-02-07 20:04:33 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:33 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:33 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.3161301612854004, 0.1737726647377258)


In [87]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :2048


STAGE:2025-02-07 20:04:33 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:34 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:34 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
STAGE:2025-02-07 20:04:34 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(1.467900276184082, 0.2783992833656157)


STAGE:2025-02-07 20:04:34 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:34 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.4834928512573242, 0.11883949307502917)


In [88]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :4096


STAGE:2025-02-07 20:04:35 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:36 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:36 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(17.618253231048584, 1.3054100301076232)


STAGE:2025-02-07 20:04:37 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:38 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:38 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(17.115893363952637, 0.6386744367091524)


In [89]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :8192


STAGE:2025-02-07 20:04:39 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:43 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:43 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(33.158440589904785, 1.5412550705832473)


STAGE:2025-02-07 20:04:43 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(32.68824815750122, 0.7578509398342703)


STAGE:2025-02-07 20:04:46 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:46 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [90]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :1024


STAGE:2025-02-07 20:04:47 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:47 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:47 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.1198248863220215, 0.09256259283461077)


STAGE:2025-02-07 20:04:47 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:48 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:48 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.8766355514526367, 0.2289112357857448)


In [91]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :2048


STAGE:2025-02-07 20:04:48 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:50 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:50 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(17.4284029006958, 0.24008303737446113)


STAGE:2025-02-07 20:04:50 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:52 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:52 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(16.99333667755127, 0.2922679659377536)


In [92]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :4096


STAGE:2025-02-07 20:04:53 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:04:56 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:04:56 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(32.55747079849243, 0.587663886625478)


STAGE:2025-02-07 20:04:57 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:00 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:00 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(32.880237102508545, 1.5573242345624294)


In [93]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :8192


STAGE:2025-02-07 20:05:01 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:07 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:07 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(61.17898225784302, 2.87885077270289)


STAGE:2025-02-07 20:05:08 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:14 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:14 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(61.0917329788208, 1.1074108789443926)


In [94]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :1024


STAGE:2025-02-07 20:05:14 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:14 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:14 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.9487357139587402, 0.2523714955430023)


STAGE:2025-02-07 20:05:15 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:15 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:15 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.5735507011413574, 0.2062227251542481)


In [95]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :2048


STAGE:2025-02-07 20:05:15 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:16 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:16 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.4594712257385254, 1.9155059486081427)


STAGE:2025-02-07 20:05:16 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:18 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:18 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(12.804210186004639, 0.7020408069990935)


In [96]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :4096


STAGE:2025-02-07 20:05:18 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:21 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:21 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(25.080430507659912, 0.8914135950988111)


STAGE:2025-02-07 20:05:21 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:24 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:24 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(24.24811601638794, 1.0595212341456772)


In [97]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :8192


STAGE:2025-02-07 20:05:24 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:29 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:29 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(47.20960855484009, 1.6093451772292156)


STAGE:2025-02-07 20:05:30 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(46.67426824569702, 1.5613369774371222)


STAGE:2025-02-07 20:05:35 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:35 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [98]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :1024


STAGE:2025-02-07 20:05:35 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:36 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:36 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(13.247509002685547, 0.5384152048918622)


STAGE:2025-02-07 20:05:37 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:38 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:38 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(12.751352787017822, 0.19957413698534765)


In [99]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :2048


STAGE:2025-02-07 20:05:39 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:41 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:41 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(25.359666347503662, 0.5181861767766804)


STAGE:2025-02-07 20:05:42 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:44 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:44 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(25.23780107498169, 0.7164086531417979)


In [100]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :4096


STAGE:2025-02-07 20:05:45 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:50 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:50 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(48.39134216308594, 1.8857420684526005)


STAGE:2025-02-07 20:05:50 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:05:55 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:05:55 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(47.80450105667114, 1.3540197470490511)


In [101]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :8192


STAGE:2025-02-07 20:05:56 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:06:05 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:06:05 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(91.73617124557495, 2.8407068895091925)


STAGE:2025-02-07 20:06:06 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(91.968092918396, 1.929563070484619)


STAGE:2025-02-07 20:06:15 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:06:15 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [102]:
def gen_pattern_replace_and_matcher_for_FM_reduce2(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)

    def forward(self,x):
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      embed = self.embed(x)
      square_of_sum = torch.sum(embed,dim=1) ** 2
      square = embed ** 2
      r_square = square[redency_part_slice] 
      n_square = square[unredency_part_slice] 
      
      sum_of_square = torch.sum(r_square,dim=0) + torch.sum(n_square,dim=1)
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [103]:
def workload_FM_reduce2(num_field, prefix,dim = 64):
  # print(f"now gen workload of wdl with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  model_ori = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  ori_traced = symbolic_trace(model_ori)
  
  model_modify = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  modify_traced = symbolic_trace(model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM_reduce2(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      )
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [104]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 32,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :1024


STAGE:2025-02-07 20:11:34 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:34 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:34 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.844719409942627, 0.07318978020407485)


STAGE:2025-02-07 20:11:34 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:35 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:35 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.8950543403625488, 0.11791012827302438)


In [105]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 32,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :2048


STAGE:2025-02-07 20:11:38 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:38 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:38 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0619401931762695, 0.11835994030207075)


STAGE:2025-02-07 20:11:38 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:38 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:38 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0939526557922363, 0.13465284570770564)


In [106]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :4096


STAGE:2025-02-07 20:11:40 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:40 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:40 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.08062744140625, 0.1380503698328539)


STAGE:2025-02-07 20:11:41 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:41 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:41 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1834907531738281, 0.21335662686396972)


In [107]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 32,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :8192


STAGE:2025-02-07 20:11:42 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:42 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:42 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.2988734245300293, 0.5169148785796551)


STAGE:2025-02-07 20:11:42 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:43 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:43 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.3092923164367676, 0.32555031254446476)


In [108]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 64,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :1024


STAGE:2025-02-07 20:11:45 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:45 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:45 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9556055068969727, 0.09794485947622888)


STAGE:2025-02-07 20:11:46 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:46 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:46 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1289572715759277, 0.10035044021492467)


In [109]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 64,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :2048


STAGE:2025-02-07 20:11:48 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:48 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:48 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.144254207611084, 0.46603278676116133)


STAGE:2025-02-07 20:11:48 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:48 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:48 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1577343940734863, 0.1289890097893931)


In [110]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 64,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :4096


STAGE:2025-02-07 20:11:50 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:50 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:50 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
STAGE:2025-02-07 20:11:51 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(1.4400434494018555, 0.42019236727810494)


STAGE:2025-02-07 20:11:51 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:51 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.9150476455688477, 0.13524026996947214)


In [111]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 64,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :8192


STAGE:2025-02-07 20:11:52 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:54 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:54 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(11.83910608291626, 0.3466949188975832)


STAGE:2025-02-07 20:11:54 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:11:55 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:11:55 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(10.040671825408936, 15.84861094403891)


In [112]:
gen_and_test(num_field = 34,prefix = 29, batch = 1024, dim = 32,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 32, num_field: 34, prefix: 29, batch :1024


STAGE:2025-02-07 20:13:12 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:13:12 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:12 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.928804874420166, 0.07339051841768196)


STAGE:2025-02-07 20:13:12 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:13:12 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:12 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9196305274963379, 0.09307828497071569)


In [113]:
gen_and_test(num_field = 34,prefix = 29, batch = 2048, dim = 32,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 32, num_field: 34, prefix: 29, batch :2048


STAGE:2025-02-07 20:13:15 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:13:15 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:15 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9540772438049316, 0.11285542620385058)


STAGE:2025-02-07 20:13:15 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:13:15 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:15 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0531353950500488, 0.16871785311991516)


In [114]:
gen_and_test(num_field = 34,prefix = 29, batch = 4096, dim = 32,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 32, num_field: 34, prefix: 29, batch :4096


STAGE:2025-02-07 20:13:17 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:13:17 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:17 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.2390708923339844, 0.21112961908329453)


STAGE:2025-02-07 20:13:17 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:13:18 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:18 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1996960639953613, 0.18160925480401602)


In [115]:
gen_and_test(num_field = 34,prefix = 29, batch = 8192, dim = 32,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 32, num_field: 34, prefix: 29, batch :8192


STAGE:2025-02-07 20:13:19 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:13:19 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:19 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
STAGE:2025-02-07 20:13:19 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(1.6350722312927246, 0.2452254744810034)


STAGE:2025-02-07 20:13:20 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:20 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(7.730302810668945, 11.33169947265742)


In [116]:
gen_and_test(num_field = 34,prefix = 29, batch = 1024, dim = 64,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :1024


STAGE:2025-02-07 20:13:22 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:13:22 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:22 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0845088958740234, 0.07830757695046486)


STAGE:2025-02-07 20:13:22 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:13:22 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:22 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1223959922790527, 0.20863201036149803)


In [117]:
gen_and_test(num_field = 34,prefix = 29, batch = 2048, dim = 64,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :2048


STAGE:2025-02-07 20:13:25 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:13:25 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:25 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1208415031433105, 0.19271494508643627)


STAGE:2025-02-07 20:13:26 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:13:26 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:26 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1440587043762207, 0.17530421638980442)


In [124]:
gen_and_test(num_field = 34,prefix = 29, batch = 4096, dim = 64,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :4096


STAGE:2025-02-07 20:14:57 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:14:58 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:14:58 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(9.788069725036621, 5.194651737406275)


STAGE:2025-02-07 20:14:58 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:14:59 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:14:59 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(10.40283203125, 0.22005791097399197)


In [119]:
gen_and_test(num_field = 34,prefix = 29, batch = 8192, dim = 64,workload_func = workload_FM_reduce2)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :8192


STAGE:2025-02-07 20:13:31 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:13:33 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:33 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(19.318530559539795, 22.69540414806086)


STAGE:2025-02-07 20:13:34 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:13:36 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:13:36 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(21.52268409729004, 2.0825404998049635)


In [5]:
def gen_pattern_replace_and_matcher_for_FM_reduce(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)

    def forward(self,x):
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      embed = self.embed(x)
      redency_embed = embed[redency_part_slice] 
      unredency_embed = embed[unredency_part_slice] 
      redency_embed_sum = torch.sum(redency_embed,dim=0)
      unredency_embed_sum = torch.sum(unredency_embed,dim=1)
      square_of_sum = (redency_embed_sum + unredency_embed_sum) ** 2
      square = embed ** 2
      r_square = square[redency_part_slice] 
      n_square = square[unredency_part_slice] 
      
      sum_of_square = torch.sum(r_square,dim=0) + torch.sum(n_square,dim=1)
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [6]:
def workload_FM_reduce(num_field, prefix,dim = 64):
  # print(f"now gen workload of wdl with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  model_ori = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  ori_traced = symbolic_trace(model_ori)
  
  model_modify = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  modify_traced = symbolic_trace(model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM_reduce(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      )
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [8]:
def genWorkload(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64):
  ori_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/fm_single/fm_{batch}_{num_field}_{prefix}_{dim}_ori.onnx'
  modify_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/fm_single/fm_{batch}_{num_field}_{prefix}_{dim}_modify.onnx'
  ori, modify = workload_FM_reduce(num_field,prefix,dim)
  torch.onnx.export(ori,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  ori_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )
  torch.onnx.export(modify,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  modify_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )

In [9]:
dims= [32]
batches = [1024,2048,4096]
num_field_and_prefixs = [(34 * 5,29*5),(22 * 5,10 * 5)]

In [ ]:
dims= [32]
batches = [1024,2048,4096]
num_field_and_prefixs = [(34 * 5,29*5),(22 * 5,10 * 5)]
for dim in dims:
  for batch in batches:
    for num_field,prefix in num_field_and_prefixs:
      genWorkload(num_field=num_field,prefix=prefix,batch=batch,dim=dim)

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symboli

In [127]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 32,workload_func = workload_FM_reduce)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :1024


STAGE:2025-02-07 20:16:05 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:16:05 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:16:05 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9169554710388184, 0.07943132970353872)


STAGE:2025-02-07 20:16:06 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:16:06 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:16:06 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0442233085632324, 0.12344684284357754)


In [128]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 32,workload_func = workload_FM_reduce)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :2048


STAGE:2025-02-07 20:16:47 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:16:47 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:16:47 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0610270500183105, 0.1236391072495735)


STAGE:2025-02-07 20:16:47 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:16:47 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:16:47 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1726069450378418, 0.19102155517316532)


In [129]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32,workload_func = workload_FM_reduce)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :4096


STAGE:2025-02-07 20:16:49 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:16:49 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:16:49 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.092071533203125, 0.08844105250318535)


STAGE:2025-02-07 20:16:50 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:16:50 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection


(1.1979436874389648, 0.13238038538929686)


STAGE:2025-02-07 20:16:50 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [132]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 32,workload_func = workload_FM_reduce)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :8192


STAGE:2025-02-07 20:16:58 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:16:58 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:16:58 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.4666962623596191, 0.1865717113730625)


STAGE:2025-02-07 20:16:59 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:16:59 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:16:59 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.3505244255065918, 0.18103512445009073)


In [131]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 64,workload_func = workload_FM_reduce)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :1024


STAGE:2025-02-07 20:16:55 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:16:55 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:16:55 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.964806079864502, 0.07223012596000444)


STAGE:2025-02-07 20:16:55 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:16:55 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:16:55 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0060453414916992, 0.1514336068112243)


In [133]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 64,workload_func = workload_FM_reduce)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :2048


STAGE:2025-02-07 20:17:01 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:17:01 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:17:01 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0033392906188965, 0.08831823230934788)


STAGE:2025-02-07 20:17:02 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:17:02 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:17:02 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0421347618103027, 0.11528772626547834)


In [134]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 64,workload_func = workload_FM_reduce)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :4096


STAGE:2025-02-07 20:17:04 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:17:04 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:17:04 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.7316913604736328, 0.18753690383164212)


STAGE:2025-02-07 20:17:04 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:17:05 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:17:05 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.1911191940307617, 0.11400780247186049)


In [135]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 64,workload_func = workload_FM_reduce)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :8192


STAGE:2025-02-07 20:17:06 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:17:08 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:17:08 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(14.165246486663818, 3.3072387934396374)


STAGE:2025-02-07 20:17:08 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:17:09 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:17:09 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(14.394018650054932, 0.5530460178931662)


In [136]:
#
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :8192


STAGE:2025-02-07 20:17:26 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:17:32 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:17:32 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(58.990938663482666, 2.01896630330225)


STAGE:2025-02-07 20:17:33 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:17:39 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:17:39 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(57.015204429626465, 1.5228789374077678)


In [137]:
#
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :4096


STAGE:2025-02-07 20:17:49 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:17:54 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:17:54 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(47.62192249298096, 1.2350552644647905)


STAGE:2025-02-07 20:17:54 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:17:59 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:17:59 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(47.41093635559082, 0.9631288245145697)


In [139]:
gen_and_test(num_field = 100,prefix = 80, batch = 4096, dim = 32,workload_func = workload_FM_reduce)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 80, batch :4096


STAGE:2025-02-07 20:34:45 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:34:46 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:34:46 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(15.614936351776123, 0.41849366118071885)


STAGE:2025-02-07 20:34:47 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:34:48 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:34:48 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(15.272209644317627, 0.8856696721238677)


In [5]:
def gen_pattern_replace_and_matcher_for_FM_pow(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  match_func = None,batch = 4096
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)

    def forward(self,x):
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      embed = self.embed(x)
      square_of_sum = torch.sum(embed,dim=1) ** 2
      # square = embed ** 2
      r_embed = embed[redency_part_slice] 
      n_embed = embed[unredency_part_slice] 
      r_square = r_embed ** 2
      n_square = n_embed ** 2
      square = torch.concat([r_square.repeat(batch,1,1),n_square],dim = 1)
      sum_of_square = torch.sum(square,dim=1)
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [6]:
def workload_FM_pow(num_field, prefix,dim = 64,batch = 4096):
  # print(f"now gen workload of wdl with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  model_ori = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  ori_traced = symbolic_trace(model_ori)
  
  model_modify = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  modify_traced = symbolic_trace(model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM_pow(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      batch = batch)
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [7]:
def genWorkload(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64):
  ori_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/fm_pow/fm_{batch}_{num_field}_{prefix}_{dim}_ori.onnx'
  modify_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/fm_pow/fm_{batch}_{num_field}_{prefix}_{dim}_modify.onnx'
  ori, modify = workload_FM_pow(num_field,prefix,dim,batch = batch)
  torch.onnx.export(ori,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  ori_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )
  torch.onnx.export(modify,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  modify_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )

In [8]:
dims= [32]
batches = [1024,2048,4096]
num_field_and_prefixs = [(34 * 5,29*5),(22 * 5,10 * 5)]
for dim in dims:
  for batch in batches:
    for num_field,prefix in num_field_and_prefixs:
      genWorkload(num_field=num_field,prefix=prefix,batch=batch,dim=dim)

/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: Us

In [150]:
def gen_and_test_pow(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_FM_pow):
  def run(model):
    traced_model = torch.jit.trace(model, torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long))
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = 0
    total_time = []
    t = torch.randint(low=0, high=88, size=(batch ,num_field), dtype=torch.long)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        with record_function("model_inference"):
          for i in range(100):
              start_time = time.time()  # 开始计时
              with torch.no_grad():
                soutput = compiled_model(t)
              end_time = time.time()  # 结束计时
              
              # 计算并打印函数执行所需的时间
              elapsed_time = end_time - start_time
              total_time.append(elapsed_time * 1000)
    print(calculate_mean_and_variance_manual(total_time))
  print(f"now gen workload of fm with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim,batch = batch)
  run(ori)
  run(modify)

In [151]:
gen_and_test_pow(num_field = 100,prefix = 80, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 80, batch :4096


STAGE:2025-02-07 20:41:05 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:41:07 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:41:07 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(16.88987970352173, 0.6170826296681753)


STAGE:2025-02-07 20:41:07 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:41:09 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:41:09 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(17.24506378173828, 0.7780305803862575)


In [152]:
gen_and_test_pow(num_field = 100,prefix = 80, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 100, prefix: 80, batch :4096


STAGE:2025-02-07 20:41:21 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:41:24 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:41:24 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(30.05770444869995, 1.0351907103256508)


STAGE:2025-02-07 20:41:24 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:41:29 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:41:29 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(42.503089904785156, 3.0521515921918763)


In [153]:
gen_and_test_pow(num_field = 100,prefix = 80, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 80, batch :4096


STAGE:2025-02-07 20:43:15 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:43:17 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:43:17 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(15.491724014282227, 0.26896194412984187)


STAGE:2025-02-07 20:43:17 2636347:2636347 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:43:19 2636347:2636347 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:43:19 2636347:2636347 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(16.30260944366455, 0.6717198715477934)


In [4]:
def gen_pattern_replace_and_matcher_for_FM_embed(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  match_func = None,batch = 4096
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)

    def forward(self,x):
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      #r_embed = embed[redency_part_slice] 
      #n_embed = embed[unredency_part_slice] 
      r_x = x[redency_part_slice] 
      n_x = x[unredency_part_slice] 
      embed_r = self.embed(r_x)
      embed_n = self.embed(n_x)
      x = torch.concat([embed_r.repeat(batch,1,1),embed_n],dim = 1)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [5]:
def workload_FM_embed(num_field, prefix,dim = 64,batch = 4096):
  # print(f"now gen workload of wdl with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  model_ori = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  ori_traced = symbolic_trace(model_ori)
  
  model_modify = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  modify_traced = symbolic_trace(model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM_embed(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      batch = batch)
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [6]:
def genWorkload(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64):
  ori_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/fm_emb/fm_{batch}_{num_field}_{prefix}_{dim}_ori.onnx'
  modify_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/fm_emb/fm_{batch}_{num_field}_{prefix}_{dim}_modify.onnx'
  ori, modify = workload_FM_embed(num_field,prefix,dim,batch = batch)
  torch.onnx.export(ori,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  ori_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )
  torch.onnx.export(modify,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  modify_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )

In [7]:
dims= [32]
batches = [1024,2048,4096]
num_field_and_prefixs = [(34 * 5,29*5),(22 * 5,10 * 5)]
for dim in dims:
  for batch in batches:
    for num_field,prefix in num_field_and_prefixs:
      genWorkload(num_field=num_field,prefix=prefix,batch=batch,dim=dim)

/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: Us

In [8]:
def gen_pattern_replace_and_matcher_for_FM_full(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  match_func = None,batch = 4096
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)

    def forward(self,x):
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      #r_embed = embed[redency_part_slice] 
      #n_embed = embed[unredency_part_slice] 
      r_x = x[redency_part_slice] 
      n_x = x[unredency_part_slice] 
      embed_r = self.embed(r_x)
      embed_n = self.embed(n_x)
      embed = torch.concat([embed_r.repeat(batch,1,1),embed_n],dim = 1)
      r_embed = embed[redency_part_slice] 
      n_embed = embed[unredency_part_slice] 
      r_square = r_embed ** 2
      n_square = n_embed ** 2
      square = torch.concat([r_square.repeat(batch,1,1),n_square],dim = 1)
      redency_embed = embed[redency_part_slice] 
      unredency_embed = embed[unredency_part_slice] 
      redency_embed_sum = torch.sum(redency_embed,dim=0)
      unredency_embed_sum = torch.sum(unredency_embed,dim=1)
      square_of_sum = (redency_embed_sum + unredency_embed_sum) ** 2
      r_square = square[redency_part_slice] 
      n_square = square[unredency_part_slice] 
      
      sum_of_square = torch.sum(r_square,dim=0) + torch.sum(n_square,dim=1)
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [9]:
def workload_FM_full(num_field, prefix,dim = 64,batch = 4096):
  # print(f"now gen workload of wdl with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  model_ori = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  ori_traced = symbolic_trace(model_ori)
  
  model_modify = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  modify_traced = symbolic_trace(model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM_full(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      batch = batch)
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [10]:
def genWorkload(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64):
  ori_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/fm_full/fm_{batch}_{num_field}_{prefix}_{dim}_ori.onnx'
  modify_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/fm_full/fm_{batch}_{num_field}_{prefix}_{dim}_modify.onnx'
  ori, modify = workload_FM_full(num_field,prefix,dim,batch = batch)
  # torch.onnx.export(ori,               # 模型 being run
  #                 torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
  #                 ori_model_name,        # 导出文件的文件名
  #                 export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
  #                 opset_version=10,   # ONNX版本
  #                 do_constant_folding=True,  # 是否执行常量折叠以优化模型
  #                 input_names = ['input'],   # 输入的名称
  #                 output_names = ['output'], # 输出的名称
  #                 )
  torch.onnx.export(modify,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  modify_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )

In [11]:
dims= [32]
batches = [1024,2048,4096]
num_field_and_prefixs = [(34 * 5,29*5),(22 * 5,10 * 5)]
for dim in dims:
  for batch in batches:
    for num_field,prefix in num_field_and_prefixs:
      genWorkload(num_field=num_field,prefix=prefix,batch=batch,dim=dim)

/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: Us